## Summary

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to do streaming responses with Instructor. It's also notable that this combo (LLM + Instructor) can create structured synthetic data.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [22]:
!pip install --quiet instructor groq openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY - get a key from https://platform.openai.com/api-keys

GROQ_API_KEY - get a key from https://console.groq.com/keys

In [23]:
import openai
import groq
import instructor
from pydantic import BaseModel, Field
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass

if not os.environ.get('OPENAI_API_KEY') or not os.environ.get('GROQ_API_KEY'):
    raise ValueError("Both OPENAI_API_KEY and GROQ_API_KEY environment variables must be set and non-empty. Read the text in the notebook (above this block) for more info.")

Now to the cool stuff...

In [24]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class User(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    gender: str = Field(description="The gender of the user")
    city: str = Field(description="The city of the user")
    state: str = Field(description="The state of the user")
    country: str = Field(description="The country of the user")
    backstory: str = Field(description="A backstory about the user")

user_stream = client.chat.completions.create_partial(   # create_partial is what does the asynchronous streaming (usually it's "create")
    model="gpt-4o" if inference_provider == "openai" else "llama-3.1-70b-versatile",
    messages=[
        {"role": "user", "content": "Create 1 synthetic user"},   # ask the LLM to make up some data
    ],
    response_model=User,
    stream=True,
)

# below is some overly complicated code to print the results...really just a prettier version of:
# for user in user_stream:
#     print(user)

previous_output = None
for user in user_stream:
    # this loops with every token
    user_dict = user.model_dump()
    current_output = ", ".join(f"{key}: {value}" for key, value in user_dict.items() if value is not None)
    if current_output != previous_output:
        print(current_output)   # only print when there's a complete new value i.e. all tokens have arrived
        previous_output = current_output




name: 
name: Alex
name: Alex Johnson
name: Alex Johnson, age: 29
name: Alex Johnson, age: 29, gender: 
name: Alex Johnson, age: 29, gender: Non
name: Alex Johnson, age: 29, gender: Non-b
name: Alex Johnson, age: 29, gender: Non-binary
name: Alex Johnson, age: 29, gender: Non-binary, city: 
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: 
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas, country: 
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas, country: USA
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas, country: USA, backstory: 
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas, country: USA, backstory: Alex
name: Alex Johnson, age: 29, gender: Non-binary, city: Austin, state: Texas, country: USA, backstory: Alex is
na